# Rendu projet Final Guillaume TRICHARD
## Détéction de la marque d'une voiture en fonction de son logo

J'importe des images de logo de voiture en fonction d'une recherche google

In [2]:
from bs4 import BeautifulSoup 
import requests
import urllib2
import json
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy
import os, sys


### Importation des images depuis GOOGLE 

Pour me créer un dataset d'une dixaine d'images de logo de différentes marques de voiture je sniff les premières images de google image grâce a une image et un texte. 
Afin d'avoir des résultats pertinents j'ai choisi pour chaque marque de voiture une image de son logo qui ressemble beaucoup a ce que l'on pourrait obtenir en prenant le logo d'une voiture dans la rue. 
Donc pour chacune des marques de voiture je créé un nouveau répertoire du nom de la marque et je récupère une dixaine d'images. 

Merci à Kevin pour son aide sur le sniffer 

In [2]:
#récupération des images de logo de différentes marques de voitures (merci kevin de l'aide apporté)
class Type:
    count = 0
    def __init__(self,name,option,img):
        self.query = name
        self.img = img
        self.option = option
        self.id = Type.count
        Type.count = Type.count +1

def get_soup(url,header):
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=header)),'html.parser')


types = []

types.append(Type("porsche", "", "CAESlQEJqdkHiSk8z-8aiQELEKjU2AQaAggLDAsQsIynCBpiCmAIAxIo7xbwFoEMxQynDOoWwQuoF7YXxAzdPOE8uT-2PbUrvD3jKaUh2jy3PRowewiOWS2jjNkUm3M0SbXXa6bK4i5RwzZrpJItrkr-4FEukdMxqdLPYxIwy8Rp1-_1KIAQMCxCOrv4IGgoKCAgBEgRy2_19cDA"))
types.append(Type("renault", "", "CAESvQEJjLMwSgRL4sgasQELEKjU2AQaBAgDCAoMCxCwjKcIGmIKYAgDEiiiBtACuAe7B6wdqx25B6wSpQehBuc2qTajNLo_1oDSUPrs_1pTS-KLooGjCM15PcD_1QG-HzQ5tGGzymP7cWlvBWLhWGaXTcmsUe5Ug3awMbz_1H5qzMNMC4LHgQUgBAwLEI6u_1ggaCgoICAESBAPE2ekMCxCd7cEJGh0KGwoIZ3JhcGhpY3PapYj2AwsKCS9tLzAyMXNkZww-djc3qn1PWaxS0tkR643fGPunuNi6KTP4FGIAQMCxCOrv4IGgoKCAgBEgRqp3GhDAsQne3BCRpQChkKBmVtYmxlbdqliPYDCwoJL20vMDF3azljChgKBWNyZXN02qWI9gMLCgkvbS8wM2p3c18KGQoHcG9yc2NoZdqliPYDCgoIL20vMDYyczQM"))
types.append(Type("citroen", "", "CAESlwEJr4jHQIQFhtwaiwELEKjU2AQaBAgACAMMCxCwjKcIGmIKYAgDEijoAdwBoATHDqEE_1xeEGNsB-xftBbY25Ca6Nqw2qjbxMqEosja1Nrc2GjAhFNOQi9B2oRzQ-MlXzE6DODYrzTvp7uMO05hpduI-MW4De_1NSa0jBF_1JzdC_1orSEgBAwLEI6u_1ggaCgoICAESBN4S1BcM"))
types.append(Type("peugeot", "", "CAESlAEJdYxS5kBeUkgaiAELEKjU2AQaBAgBCAkMCxCwjKcIGl8KXQgDEiVIxwEbiwcaqB3KAckBuwLAAYo0-zOlKawh7D2GNI01iieCNPkzGjAaKGgMqtAPIoog3yer_1y2X4zBceGOUQee742q7pLY40Ql5m7tQpsvUWvGA7QMvMdkgBAwLEI6u_1ggaCgoICAESBAotWoIM"))
types.append(Type("bugatti", "", "CAES1AEJnPTh8X_1Ah8IayAELEKjU2AQaAggIDAsQsIynCBpiCmAIAxIo9wO2Ap0K4AG1AswShAqDCrcCzRLGP649rz2aNMc_1sj25P-A8mTS8PRowiAJrxfJXLCsfV-SXF2YDFM0g9b1rHzCP_1MtltqRjRC11tJ1suj8-585UNLnqZXCMIAQMCxCOrv4IGgoKCAgBEgSBrBoGDAsQne3BCRo2ChsKCXRyYWRlbWFya9qliPYDCgoIL20vMDdkMGYKFwoEb3ZhbNqliPYDCwoJL20vMDZnNThiDA"))
types.append(Type("ferrari", "", "CAESkwEJU3JlxMq4c7sahwELEKjU2AQaAggLDAsQsIynCBpgCl4IAxImgglGhwmDCbQKyBTGFP0DYfsIsDq5P908iCrCPbk94ynHP4on5SkaMHE1CcfTQOxzsjA9CTYHG3zzzbxD5kaUYGCQTVQCKsB9JcJ5ydc4rV1cuu8TOtByCyAEDAsQjq7-CBoKCggIARIEAl7G6ww"))
types.append(Type("alhpaRomeo", "", "CAEShwEJTm5RND9xouQafAsQsIynCBpgCl4IAxImrwSiDK8MbrMEsgw9rATFA6cMryv4Iqk3hCCIM_1IhnzmeObUrsCsaMG41mIfTYJMkQ5La9GgrVyOMU7cQgAD_1HfpOJkyfoUAYnuAftFVI2LbPeZYdOtS3mSAEDAsQjq7-CBoKCggIARIEsjyttQw"))
types.append(Type("audi", "", "CAESlwEJbyjGH6Cuxz0aiwELEKjU2AQaBAgACAEMCxCwjKcIGmIKYAgDEiiwDK8MwwO1DJYXjAyyDJELvAzXBIA3sCizM5Yg6SaBN-M2yzO9M_1ogGjCZFJVa2xR-OIrlZ5E5TREwWAe7FTEFh7O5akfpG3JovZSClwZ4soP1i3kkseCaZBggBAwLEI6u_1ggaCgoICAESBIevlWsM"))
types.append(Type("mercedes", "", "CAESkwEJ0oJ6IcYLMhcahwELEKjU2AQaAggIDAsQsIynCBpgCl4IAxImzwGyAs4B4AEptgL_1CLQC_1QgovTPGP7s_1rj2aNOE8zzLHP4Q04DwaMHQjh2Dq1yw_1u6X7tVllcqkRBHssbwcgkIU02ZBrDB6HUiQCl9c651BSwovxCusxmCAEDAsQjq7-CBoKCggIARIEieoA5ww"))

def download():
    for currentType in range(0,len(types)):
        
        query= types[currentType].query.split()
        query='+'.join(query)
        url="https://www.google.fr/search?safe=off&sa=G&hl=fr&tbs=simg:"+types[currentType].img+"&q="+query+"&tbm=isch&ved=0ahUKEwi71dvR57DSAhXSSxoKHbBFARgQsw4ILA&biw=1745&bih=841"
        print url
        # Spécification du répertoire de déstination pour stocker les images 
        DIR="marques"
        DIR2 = types[currentType].query
        header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}
        soup = get_soup(url,header) 


        ActualImages=[]
        for a in soup.find_all("div",{"class":"rg_meta"}):
            link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
            ActualImages.append((link,Type))

        #si le dossier qui contiens les logo d'une marque n'est pas créé alors je l'ajoute
        if not os.path.exists(DIR):
                   os.mkdir(DIR)
        if not os.path.exists(DIR+"/"+DIR2):
                   os.mkdir(DIR+"/"+DIR2)
 
        cntr = 0
        for i , (img , Type) in enumerate( ActualImages):
            if i > 20:
                break;
            try:
                req = urllib2.Request(img, headers={'User-Agent' : header})
                raw_img = urllib2.urlopen(req).read()
                if len(Type)==3:
                    cntr = cntr + 1
                    f = open(os.path.join(DIR ,DIR2+"/"+ str(cntr)+".jpg"), 'wb')
                    f.write(raw_img)
                    f.close()
                    size = 128,128
                   
                    im = Image.open(DIR+"/"+DIR2+"/"+ str(cntr)+".jpg")
                    #im.thumbnail(size, Image.ANTIALIAS)
                    im.save(DIR+"/"+DIR2+"/"+ str(cntr)+".jpg", "JPEG")



            except Exception as e:
                print "could not load : "+img
                print e
download()

https://www.google.fr/search?safe=off&sa=G&hl=fr&tbs=simg:CAESlQEJqdkHiSk8z-8aiQELEKjU2AQaAggLDAsQsIynCBpiCmAIAxIo7xbwFoEMxQynDOoWwQuoF7YXxAzdPOE8uT-2PbUrvD3jKaUh2jy3PRowewiOWS2jjNkUm3M0SbXXa6bK4i5RwzZrpJItrkr-4FEukdMxqdLPYxIwy8Rp1-_1KIAQMCxCOrv4IGgoKCAgBEgRy2_19cDA&q=porsche&tbm=isch&ved=0ahUKEwi71dvR57DSAhXSSxoKHbBFARgQsw4ILA&biw=1745&bih=841
could not load : http://i1.wp.com/www.inautonews.com/wp-content/uploads/2015/02/porsche-badge.jpg?resize=350%2C200
HTTP Error 400: Bad Request
could not load : http://i0.wp.com/www.inautonews.com/wp-content/uploads/2012/08/porsche-panamera-badge.jpg?resize=350%2C200
HTTP Error 400: Bad Request
https://www.google.fr/search?safe=off&sa=G&hl=fr&tbs=simg:CAESvQEJjLMwSgRL4sgasQELEKjU2AQaBAgDCAoMCxCwjKcIGmIKYAgDEiiiBtACuAe7B6wdqx25B6wSpQehBuc2qTajNLo_1oDSUPrs_1pTS-KLooGjCM15PcD_1QG-HzQ5tGGzymP7cWlvBWLhWGaXTcmsUe5Ug3awMbz_1H5qzMNMC4LHgQUgBAwLEI6u_1ggaCgoICAESBAPE2ekMCxCd7cEJGh0KGwoIZ3JhcGhpY3PapYj2AwsKCS9tLzAyMXNkZww-djc3qn1PWaxS0tkR643fGPunuNi6KTP4FGI

Je fait une repasse a la main une fois que tout les logos sont téléchargé car il y a malgré tout des logos qui ne correspondent pas du tout a ce que je recherche. De plus il y a certaines images qui ne sont même pas lisibles

### Réduction de la taille des images 

Une fois que toutes les images sont téléchargé je réduit la taille de chacune d'elles afin d'augmenter la vitesse de traitement. 
En effet quand mon dataset sera bien garnis le nombre de comparaison sera vraiment long si je ne réduit pas la taille. 
De plus cela me permet d'avoir une uniformité et donc d'augmenter la réussite de mon programme


In [3]:
#réduire la taille des images pour augmenter la vitesse d'analyse 

size = 128,128
for filename in os.listdir("marques"):#je fait une boucle sur tout les répertoires des marques 
     print(filename)
     result = 0
     distance = 0
     for filename2 in os.listdir(os.path.join("marques", filename)):
         path = "marques/"+filename+"/"+filename2
         im = Image.open(path)
         im.thumbnail(size, Image.ANTIALIAS)
         im.save(path, "JPEG")

audi
bugatti
citroen
ferrari
mercedes
peugeot
porsche
renault


### Duplication des images en noir et blanc 

Une fois que toutes mes images sont de même taille je refait un traitement qui consiste a dupliquer chaqune d'elle en noir et blanc afin de faire abstraction des couleurs qui peuvent parfois être parasite (couleur de carrosserie, éclairage non naturel...)
Cela me permet aussi d'enrichir mon dataset

In [4]:
#convert image en noir et blanc
for filename in os.listdir("marques"):
    
    print(filename)
    result = 0
    distance = 0
    for filename2 in os.listdir(os.path.join("marques", filename)): 
        path = "marques/"+filename+"/"+filename2
        image_file = Image.open(path) # open colour image
        image_file = image_file.convert('1') # convert image to black and white
        path2 = "marques/"+filename+"/n"+filename2
        image_file.save(path2)


audi
bugatti
citroen
ferrari
mercedes
peugeot
porsche
renault


### Création des differentes fonctions de comparaison d'images 

J'utilise la librairie OpenCV pour comparer mes images. 
J'utilise 3 algorythmes de OpenCV a savoir ORB, sift et surf. 
Pour me faciliter le travail j'ai créer trois fonctions qui prennent en paramètre 2 images et qui me retourne un numéro qui fait office de score de ressemblance entre les deux images.
A mon sens utiliser 3 algorythmes différent augmente les chances de réussite de détéction de la bonne marque mais en revanche cela ralenti l'analyse surtout sift.

In [5]:
img1 = cv2.imread('1.png',0)          # queryImage
img2 = cv2.imread('2.png',0) # trainImage
img3 = cv2.imread('3.png',0)
imgcitroen = cv2.imread('citroen.jpg',0)
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)
kpc, desc = orb.detectAndCompute(imgcitroen,None)


# create ORB object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

sift = cv2.xfeatures2d.SIFT_create()
def CompareWithBf(image1, image2):
    kp1, des1 = orb.detectAndCompute(image1,None)
    kp2, des2 = orb.detectAndCompute(image2,None)
    matches = bf.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)
    score = 0
    for n in matches:
        if n.distance < 40:
            score += n.distance
        else:
            break
    return score




def CompareWithSift(image1, image2):# fonction de comparaison de deux images avec l'algo sift
    kp1, des11 = sift.detectAndCompute(image1,None)
    kp2, des22 = sift.detectAndCompute(image2,None)
    matches = bf.knnMatch(des11,des22, k=2)
    good = []
    score = 0
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    score = score + len(good)
    return score





import numpy
def CompareWithSurf(img1, img2, img1_features=None, img2_features=None):# fonction de comparaison de deux images avec l'algo surf
    """Given two images, returns the matches"""
    
    detector = cv2.xfeatures2d.SURF_create(3200)
    
    matcher = cv2.BFMatcher(cv2.NORM_L2)

    if img1_features is None:
        kp1, desc1 = detector.detectAndCompute(img1, None)
    else:
        kp1, desc1 = img1_features

    if img2_features is None:
        kp2, desc2 = detector.detectAndCompute(img2, None)
    else:
        kp2, desc2 = img2_features

    raw_matches = matcher.knnMatch(desc1, trainDescriptors=desc2, k=2)
    kp_pairs = filter_matches(kp1, kp2, raw_matches)
    return len(kp_pairs)


def filter_matches(kp1, kp2, matches, ratio=0.75):
    #je filtre les matches pour garder que les plus pertinants 
    mkp1, mkp2 = [], []
    for m in matches:
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            m = m[0]
            mkp1.append(kp1[m.queryIdx])
            mkp2.append(kp2[m.trainIdx])
    kp_pairs = zip(mkp1, mkp2)
    return kp_pairs


### Test de detection d'un logo

Dans la section ci-dessous je compare tout les dataset des différentes marques avec une image spécifique.
Je compare l'image avec tout mon dataset à travers mes trois fonctions afin de multiplier les sources de comparaison et présiser au mieux le score de chacune des marques.

In [8]:
img1 = cv2.imread('1.png',0)          # queryImage
img2 = cv2.imread('2.png',0) # trainImage
img3 = cv2.imread('ferrari.jpg',0)
imgcitroen = cv2.imread('citroen.jpg',0)
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)
kpc, desc = orb.detectAndCompute(imgcitroen,None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
sift = cv2.xfeatures2d.SIFT_create()

# create ORB object
finalResult = []

print "Compare with ORB"
scoreTotal = 0
scoreArray = []
for filename in os.listdir("marques"):
    result = 0
    distance = 0
    for filename2 in os.listdir(os.path.join("marques", filename)):
        path = "marques/"+filename+"/"+filename2
        image = cv2.imread(path,0)
        
        result += CompareWithBf(image, img3)

    scoreTotal = scoreTotal + result*100 / len(os.walk("marques/"+filename+"/").next()[2])
    scoreArray.append(result*100 / len(os.walk("marques/"+filename+"/").next()[2]))
loop = 0
for filename in os.listdir("marques"):
    print(filename)
    print scoreArray[loop]*100/scoreTotal
    finalResult.append(scoreArray[loop]*100/scoreTotal)
    loop = loop + 1


print "Compare with sift"
scoreTotal = 0
scoreArray = []
bf = cv2.BFMatcher()
for filename in os.listdir("marques"):
    result = 0
    distance = 0
    for filename2 in os.listdir(os.path.join("marques", filename)):
        path = "marques/"+filename+"/"+filename2
        image = cv2.imread(path,0)
        
        result += CompareWithSift(image, img3)

    scoreTotal = scoreTotal + result*100 / len(os.walk("marques/"+filename+"/").next()[2])
    scoreArray.append(result*100 / len(os.walk("marques/"+filename+"/").next()[2]))
loop = 0
for filename in os.listdir("marques"):
    print(filename)
    finalResult[loop] = finalResult[loop] + scoreArray[loop]*100/scoreTotal
    print scoreArray[loop]*100/scoreTotal
    loop = loop + 1
    
    
    
    
    
print "Compare with surf"
scoreTotal = 0
scoreArray = []
bf = cv2.BFMatcher()
for filename in os.listdir("marques"):
    result = 0
    distance = 0
    for filename2 in os.listdir(os.path.join("marques", filename)):
        path = "marques/"+filename+"/"+filename2
        image = cv2.imread(path,0)
        
        result += CompareWithSurf(image, img3)

    scoreTotal = scoreTotal + result*100 / len(os.walk("marques/"+filename+"/").next()[2])
    scoreArray.append(result*100 / len(os.walk("marques/"+filename+"/").next()[2]))
loop = 0
for filename in os.listdir("marques"):
    print(filename)
    finalResult[loop] += scoreArray[loop]*100/scoreTotal
    print scoreArray[loop]*100/scoreTotal
    loop = loop + 1
    
    
    
print(finalResult)
    
    


Compare with ORB
audi
0.371935715211
bugatti
0.0
citroen
9.16771276412
ferrari
44.3924662715
mercedes
13.1596975804
peugeot
7.95299082287
porsche
0.0
renault
24.9551968459
Compare with sift
audi
12
bugatti
12
citroen
8
ferrari
47
mercedes
6
peugeot
3
porsche
3
renault
6
Compare with surf
audi
14
bugatti
4
citroen
9
ferrari
32
mercedes
17
peugeot
4
porsche
7
renault
9
[26.371935715210945, 16.0, 26.167712764118498, 123.39246627149046, 36.159697580362405, 14.952990822872795, 10.0, 39.9551968459449]
